# TP SQL - Postgres/Postgis - M1 GLET - Partie 2

La réalisation de ce TP suppose que vous ayez déjà lu et mis en oeuvre le document word d'instruction pour l'installation de l'environnement de travail.

Le TP est à rendre avant le vendredi 15 mai 17h00 par un envoi du notebook sur ma boite mail.

En cas de besoin envoyez moi un mail à n.dhuygelaere@oieau.fr ou contactez moi via skype à n.dhuygelaere.

Pour cette deuxième partie du TP, nous allons nous allons utiliser quelques fonctions de Postgis.

## I. Initialisation des variables et chargement des librairies

In [ ]:
#Cette première cellule permet de charger en mémoire toutes les librairies qui vous seront utiles
%matplotlib inline
from matplotlib import pyplot as plt
import psycopg2 as pg
import numpy as np
import pandas.io.sql as psql
import pandas as pd
import folium
from IPython.display import display
from IPython.display import HTML

#Paramètres de connexion à la base postgres
conn = pg.connect("dbname=tp_sql port=5432 user=postgres password=postgres")

#La requête suivante va nous permettre de tester le bon fonctionnement de la connexion 
# en affichant le code et le nom des 10 premiers tuples de la table sous_secteur
# La clause SQL LIMIT permet de ne retourner que les 10 premiers tuples
test = psql.read_sql("SELECT cdsoussect,lbsoussect  FROM sous_secteur LIMIT 10", conn)
test

## II. Postgis 

Pour mémoire, nous avons chargé 3 tables dans notre base de données : "sous_secteur", "obstacles" et "coursdeau". Les tables "sous_secteur" et "coursdeau" sont issues de la BD Carthage 2017. La table "obstacles" correspond à l'inventaire des obstacles à l'écoulement identifiés par le Système d'Information sur l'Eau. 

* La table sous_secteur contient des polygones
* La table coursdeau contient des lignes
* la table obstacles contient des points.

Toutes ces tables utilisent la projection lambert 93 (EPSG:2154) qui est une projection métrique. Les différents fonction de calcul que nous utiliserons donne le résulat dans l'unité de la projection de la géométrie. Nous n'auront donc pas besoin de convertir les coordonnées en projection métrique pour faire ces calculs. SI nous avions des données en WGS 84 (EPSG:4326), cette convertion serait nécessaire au risque d'avoir des résultats exprimés en degrés. 

### II.1. Calcul d'une longueur

A l'aide de la fonction **st_length** https://postgis.net/docs/ST_Length.html, calculez la longueur de la rivière (table coursdeau) "la morelle" de code (colonne cdentitehy) : H71-0400


In [ ]:
l_morelle = psql.read_sql("Votre requête ICI", conn)
l_morelle

### II.2. Calcul d'une superficie

A l'aide de la fonction **st_area** : https://postgis.net/docs/ST_Area.html, afficher les noms (lbsoussect) et les codes (cdsoussect) des sous-secteurs et calculez la superficie de tous les sous-secteurs hydrographiques (table sous_secteur) de la région hydrographique "La Dordogne" (colonne cdregionhy = P)

In [ ]:
surf_dordogne = psql.read_sql("Votre requête ICI", conn)
surf_dordogne

### II.3. Croisements géographiques

Postgis dispose d'un grand nombre de fonctions permettant de faire des croisements géographiques. La documentation officielle est disponible à cette adresse : https://postgis.net/docs/reference.html#Spatial_Relationships . Pour ce TP nous utiliserons les fonctions :
- **st_intersects** qui permet de vérifier si il y a intersection entre 2 géométries
==> la doc est ici : https://postgis.net/docs/ST_Intersects.html
- **st_contains** qui permet de vérifier si une géométrie en contient une autre
==> la doc est ici : https://postgis.net/docs/ST_Contains.html

Une explication plus détaillée des différentes relation géographique est disponible ici : https://postgis.net/docs/using_postgis_dbmanagement.html#DE-9IM ; Je pense que vous étudierez ces éléments en détail l'année prochaine dans le cadre du module postgres/postgis assuré par un autre intervenant.

Exemple : je souhaite trouver tous les obstacles à l'écoulement contenus dans la région hydrographique de la Dordogne et je les affiche ensuite sur une carte


In [ ]:
#Requete de selection

obs_dordogne = psql.read_sql(" \
    SELECT obs.cdobstecou, st_x(st_transform(obs.geom, 4326)) as x, st_y(st_transform(obs.geom, 4326)) as y \
    FROM sous_secteur ss, obstacles obs \
    WHERE ss.cdregionhy='P' \
         AND ss.geom && obs.geom \
         AND st_contains(ss.geom, obs.geom)", conn)
obs_dordogne

In [ ]:
#Affichage sur la carte
display(HTML("<h2>Carte des obstacles à l'écoulement de la région hydrographique \"La dordogne\"</h2>"))
lat_mean = np.mean(obs_dordogne.y)
lon_mean = np.mean(obs_dordogne.x)
print("Centre de la carte (lat, lon)", lat_mean, lon_mean)

map = folium.Map(
    tiles='OpenStreetMap',
    location=(lat_mean, lon_mean),
    zoom_start=7
)
folium.LayerControl().add_to(map)
map_tooltip = 'Click me!'
#Add markers on map
for lat, lon, code in zip(obs_dordogne.y, obs_dordogne.x, obs_dordogne.cdobstecou):
    folium.Marker(
        [lat, lon], 
        popup=f'<b><a target="_blank" href="http://www.sandre.eaufrance.fr/urn.php?urn=urn:sandre:donnees:ObstEcoul:FRA:code:{code}:::::html">{code}</a></b>',
        tooltip=map_tooltip
    ).add_to(map)

display(map)

1. Sur le même principe que ci-dessus, écrire la requête pour sélectionner tous les obstacles à l'écoulement de la région de la Meuse (code : B)
2. A partir de la requête du point 1, faire le même croisement en utilisant la fonction st_intersects, que constatez vous ?
3. Sélectionnez tous les cours d'eau (table coursdeau) de la région hydrographique de la Meuss (code : B) en utilisant
    * La fonction st_contains
    * La fonction st_intersects
    * Comparer les résultats

In [ ]:
#1

In [ ]:
#2

In [ ]:
#3.1

In [ ]:
#3.2

4. Trouver la liste des obstacles à l'écoulement situés sur le cours d'eau la morelle de code : H71-0400
    4.1. En faisant une sélection attributaire sur la colonne **cdentitehy** de la table **obstacles**
    4.2. En faisant un croisement géographique à l'aide de la fonction **st_intersects** sur les colonnes **geom** des tables **coursdeau** et **obstacles**. Selon vous pourquoi le résultat n'est pas le même (Vous pouvez vous aider de QGIS pour visualiser les géometries) ?
    4.3. Refaite la requête du 4.2, en faisant un buffer de 100 mètres autours des obstacles à l'écoulement à la l'aide  de la fonction **st_buffer** : https://postgis.net/docs/ST_Buffer.html
    

In [ ]:
#4.1

In [ ]:
#4.2

In [ ]:
#4.3